In [1]:
# pqd_predictions_test01_dataset05.ipynb
# This file loads the CNN model trained in pqd_cnn_test01_dataset05, 
# i.e. with the model structure same as the paper and the pqd_dataset_05.
# Then, this file loads the prediction dataset from pqd_dataset_05, and
# generates a CSV output file of the prediction score of each image.

In [2]:
# Import dependencies
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import models, utils, preprocessing
from google.colab import drive, files

In [3]:
# Connect to the Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load the trained CNN model
model_path = '/content/drive/My Drive/saved_models/pqd_cnn_test01_dataset05_model.h5'
cnn = models.load_model(model_path)

In [5]:
# Import the testing dataset for extracting the categories
testing_dataset = utils.image_dataset_from_directory(
    directory='/content/drive/My Drive/pqd_dataset_05/testing_set',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    image_size=(200, 200)
)

Found 1400 files belonging to 10 classes.


In [6]:
# Extract the categories of the data
type_names = testing_dataset.class_names
df_columns = ["image_name"] + type_names

In [7]:
# Define a pd dataframe
df = pd.DataFrame(
    columns=df_columns
)

In [8]:
# Import the prediction dataset
prediction_set_path = '/content/drive/My Drive/pqd_dataset_05/prediction_set'
prediction_set = tf.io.gfile.listdir(prediction_set_path)

In [9]:
# Iterate through the dataset, and
# make predictions with the trained CNN model.
# Log the prediction scores in the pd dataframe
count = 0
for image_name in prediction_set:
    prediction_image = utils.load_img(
        path=prediction_set_path + "/" + image_name,
        color_mode='grayscale',
        target_size=(200, 200)
    )
    prediction_image_array = preprocessing.image.img_to_array(
        prediction_image
    )
    prediction_image_array = np.array(
        [prediction_image_array]
    )
    predictions = cnn.predict(prediction_image_array)
    df_row = [image_name]
    for prediction in predictions[0]:
      df_row.append(prediction)
    df.loc[count] = df_row
    count += 1

In [10]:
# Sanity check
print(df.head())

                               image_name      flickers     harmonics  \
0                harmonics_sample0801.png  5.107307e-13  9.993467e-01   
1           sags_harmonics_sample0482.png  1.838267e-27  8.184322e-19   
2         swells_harmonics_sample0828.png  1.942991e-34  0.000000e+00   
3  interruptions_harmonics_sample0437.png  3.701537e-08  9.843490e-07   
4                harmonics_sample0183.png  7.497318e-13  9.993544e-01   

   interruptions  interruptions_harmonics  osc_transients          sags  \
0   6.528982e-13             6.258563e-04    2.507409e-06  7.813968e-11   
1   1.150339e-23             5.660992e-14    7.026301e-13  1.192655e-22   
2   0.000000e+00             9.544532e-33    1.269823e-13  0.000000e+00   
3   7.706452e-06             1.425291e-01    7.167400e-05  1.747777e-03   
4   9.553180e-13             6.135835e-04    2.368639e-06  9.441495e-11   

   sags_harmonics        spikes        swells  swells_harmonics  
0    1.266457e-05  2.624400e-11  2.829611e-1

In [11]:
# Save the pd dataframe as a CSV file.
# Download the CSV file
df.to_csv('pqd_predictions_test01_dataset05.csv') 
files.download('pqd_predictions_test01_dataset05.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>